In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import math
import random
from collections import Counter, defaultdict

#import nltk
from nltk.corpus import stopwords, movie_reviews

In [ ]:
df = pd.read_csv('/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv')
pd.to_datetime(df['date'])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.date.value_counts().sort_index()

In [ ]:
#df.text[0].split(' ')[0]
all_words = []
hashtags_in_text = []

stop = stopwords.words('english')

for i in df.text[:500]:
    for j in i.split(' '):
        if j.lower() not in stop:
            if '#' not in j and 'https' not in j:
                all_words.append(j.lower())
            elif '#' in j:
                hashtags_in_text.append(j)
        
all_words_dict = Counter(all_words)
hashtags_in_text_dict = Counter(hashtags_in_text)

In [ ]:
all_words_dict.most_common(5)

In [ ]:
hashtags_in_text_dict.most_common(5)

# Word cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def text_size(total):
    return 30 + ((total / 200) * 40)

most_common = []
str_common = ''
stopwords = set(STOPWORDS)

for word, value in all_words_dict.most_common(40):
    most_common.append(word)

str_common += ' '.join(most_common)

plt.figure(figsize = (15, 8), facecolor = None)


wordcloud = WordCloud(width = 800, height = 800, 
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10, mode="RGBA").generate(str_common)
plt.imshow(wordcloud)
plt.axis("off") 
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
def text_size(total):
    return 30 + ((total / 200) * 40)

most_common = []
str_common = ''
stopwords = set(STOPWORDS)

for word, value in hashtags_in_text_dict.most_common(40):
    most_common.append(word)

str_common += ' '.join(most_common)

plt.figure(figsize = (15, 8), facecolor = None)

wordcloud = WordCloud(width = 800, height = 800, 
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10, mode="RGBA").generate(str_common)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off") 
plt.tight_layout(pad = 0)
plt.show()

# Naive Bayes

In [ ]:
df.head(2)

In [ ]:
df.text.values[0]

In [ ]:
tweet = df['text'][0].lower()
all_words = re.findall("[a-z0-9']+", tweet)
all_words

In [ ]:
for i, j in zip(df.text.values, df.user_verified.values):
    print(i, j)
    break

# Naive Bayes

I use Naive Bayes theorem to find out whether a tweet is from a verified account or not. The following is the algorithm.

****-------------------------- UNDER CONSTRUCTION ---------------------------****

In [ ]:
def tokenize(tweet):
    tweet = tweet.lower()
    all_words = re.findall("[a-z0-9']+", tweet)
    return set(all_words)

In [ ]:
def count_UV(training_set):
    counts = defaultdict(lambda: [0, 0])
    for tweet, is_verified in training_set:
        for word in tokenize(tweet):
            if is_verified:
                counts[word][0] += 1
            else:
                counts[word][1] += 1
    return counts

In [ ]:
def word_probs(total_verified, total_non_verified, counts, k = 1.0):
    probs = []
    for word, (verified, not_verified) in counts.items():
        verified_prob = (k + verified) / (2 * k + total_verified)
        not_verified_prob = (k + not_verified) / (2 * k + total_non_verified)
        probs.append((word, verified_prob, not_verified_prob))
    return probs

In [ ]:
def verified_prob(probs, tweet):
    tweet_words = tokenize(tweet)
    log_prob_if_verified = 0.0
    log_prob_if_not_verified = 0.0
    
    for word, prob_if_verified, prob_if_not_verified in probs:
        if word in tweet_words:
            log_prob_if_verified += math.log(prob_if_verified)
            log_prob_if_not_verified += math.log(prob_if_not_verified)
        else:
            log_prob_if_verified += math.log(1.0 - prob_if_verified)
            log_prob_if_not_verified += math.log(1.0 - prob_if_not_verified)
            
    prob_if_verified = math.exp(log_prob_if_verified)
    prob_if_not_verified = math.exp(log_prob_if_not_verified)
    return prob_if_verified / (prob_if_verified + prob_if_not_verified)

In [ ]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        
    def train(self, training_set):
        num_verified = len([is_verified
                            for message, is_verified in training_set
                            if is_verified])
        num_non_verified = len(training_set) - num_verified
        
        word_counts = count_UV(training_set)
        self.word_probs = verified_prob(word_counts)